In [3]:
import pandas as pd
import tensorflow
from nltk.tokenize import regexp_tokenize, sent_tokenize
from nltk import pos_tag_sents
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import re
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import pickle
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')


#Dictionary of contractions for replacement 

contractions = {
          "ain't": "am not",
          "aren't": "are not",
          "can't": "cannot",
          "can't've": "cannot have",
          "'cause": "because",
          "could've": "could have",
          "couldn't": "could not",
          "couldn't've": "could not have",
          "didn't": "did not",
          "doesn't": "does not",
          "don't": "do not",
          "hadn't": "had not",
          "hadn't've": "had not have",
          "hasn't": "has not",
          "haven't": "have not",
          "he'd": "he would",
          "he'd've": "he would have",
          "he'll": "he will",
          "he'll've": "he will have",
          "he's": "he is",
          "how'd": "how did",
          "how'd'y": "how do you",
          "how'll": "how will",
          "how's": "how is",
          "i'd": "i would",
          "i'd've": "i would have",
          "i'll": "i will",
          "i'll've": "i will have",
          "i'm": "i am",
          "i've": "i have",
          "isn't": "is not",
          "it'd": "it had",
          "it'd've": "it would have",
          "it'll": "it will",
          "it'll've": "it will have",
          "it's": "it is",
          "let's": "let us",
          "ma'am": "madam",
          "mayn't": "may not",
          "might've": "might have",
          "mightn't": "might not",
          "mightn't've": "might not have",
          "must've": "must have",
          "mustn't": "must not",
          "mustn't've": "must not have",
          "needn't": "need not",
          "needn't've": "need not have",
          "o'clock": "of the clock",
          "oughtn't": "ought not",
          "oughtn't've": "ought not have",
          "shan't": "shall not",
          "sha'n't": "shall not",
          "shan't've": "shall not have",
          "she'd": "she would",
          "she'd've": "she would have",
          "she'll": "she will",
          "she'll've": "she will have",
          "she's": "she is",
          "should've": "should have",
          "shouldn't": "should not",
          "shouldn't've": "should not have",
          "so've": "so have",
          "so's": "so is",
          "that'd": "that would",
          "that'd've": "that would have",
          "that's": "that is",
          "there'd": "there had",
          "there'd've": "there would have",
          "there's": "there is",
          "they'd": "they would",
          "they'd've": "they would have",
          "they'll": "they will",
          "they'll've": "they will have",
          "they're": "they are",
          "they've": "they have",
          "to've": "to have",
          "wasn't": "was not",
          "we'd": "we had",
          "we'd've": "we would have",
          "we'll": "we will",
          "we'll've": "we will have",
          "we're": "we are",
          "we've": "we have",
          "weren't": "were not",
          "what'll": "what will",
          "what'll've": "what will have",
          "what're": "what are",
          "what's": "what is",
          "what've": "what have",
          "when's": "when is",
          "when've": "when have",
          "where'd": "where did",
          "where's": "where is",
          "where've": "where have",
          "who'll": "who will",
          "who'll've": "who will have",
          "who's": "who is",
          "who've": "who have",
          "why's": "why is",
          "why've": "why have",
          "will've": "will have",
          "won't": "will not",
          "won't've": "will not have",
          "would've": "would have",
          "wouldn't": "would not",
          "wouldn't've": "would not have",
          "y'all": "you all",
          "y'alls": "you alls",
          "y'all'd": "you all would",
          "y'all'd've": "you all would have",
          "y'all're": "you all are",
          "y'all've": "you all have",
          "you'd": "you had",
          "you'd've": "you would have",
          "you'll": "you you will",
          "you'll've": "you you will have",
          "you're": "you are",
          "you've": "you have"
            }

2024-07-06 21:54:17.458939: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-06 21:54:17.461751: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-06 21:54:17.509057: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-06 21:54:17.510427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-06 21:54:18.843769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [126]:
import pandas as pd 

pd.set_option('display.max_columns', None)

ics = pd.read_csv("REF 2021 Impact Case Studies - Extract - 2024-05-16.csv", encoding="latin1")

ics_results = pd.read_csv('REF 2021 Results - All - 2022-05-06.csv', encoding="latin1", skiprows=5)

ics_results.columns = ics_results.iloc[0]

ics_results = ics_results[1:]

ics_results.reset_index(drop=True, inplace=True)

ics_results.columns = ics_results.columns.astype(str)

ecs = pd.read_csv("REF 2021 Environment Data - All - 2022-06-21.csv", skiprows = 3)

ecs.columns = ecs.iloc[0]

ecs = ecs[1:]

ecs.reset_index(drop=True, inplace=True)

ref_outputs = pd.read_csv("REF 2021 Outputs - All - 2023-09-08.csv", encoding="latin1", skiprows = 3)

ref_outputs.columns = ref_outputs.iloc[0]

ref_outputs = ref_outputs[1:]

ref_outputs.reset_index(drop = True, inplace = True)

ref_outputs.head()

/tmp/ipykernel_1454231/3844787731.py:25: DtypeWarning: Columns (0,3,22,24,34,35,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  ref_outputs = pd.read_csv("REF 2021 Outputs - All - 2023-09-08.csv", encoding="latin1", skiprows = 3)


,Institution UKPRN code,Institution name,Main panel,Unit of assessment number,Unit of assessment name,Multiple submission letter,Multiple submission name,Joint submission,Output type,Title,Place,Publisher,Volume title,Volume,Issue,First page,Article number,ISBN,ISSN,DOI,Patent number,Month,Year,URL,Number of additional authors,Non-English,Interdisciplinary,Forensic science,Criminology,Propose double weighting,Is reserve output,Research group,Open access status,Citations applicable,Citation count,Cross-referral requested,Supplementary information,Delayed by COVID19,REF2ID,Incl sig material before 2014,Incl reseach process,Incl factual info about significance
0,10007760,Birkbeck College,D,26,Modern Languages and Linguistics,NaN,NaN,NaN,C,Text on Image on Text: Picturing âNatureâ ...,NaN,University of Delaware Press,Visualizing the Text From Manuscript Culture ...,NaN,NaN,NaN,NaN,978-1611496451,NaN,NaN,NaN,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Out of scope for open access requirements,NaN,NaN,NaN,NaN,NaN,9062d547-00c8-47a2-bd0c-4ea4956ffb85,0,0,0
1,10007848,University of Chester,C,14,Geography and Environmental Studies,NaN,NaN,NaN,D,"""Digital by Default"" and ""the hard to reach"": ...",NaN,NaN,Local Economy,31,7,757,NaN,NaN,0269-0942,10.1177/0269094216670938,NaN,September,2016,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Compliant,NaN,NaN,NaN,NaN,NaN,6ad5c192-a622-4468-be71-dba2570735bf,0,0,0
2,10007855,Swansea University / Prifysgol Abertawe,D,27,English Language and Literature,NaN,NaN,NaN,C,"""Stand not on that brink!"": Byron, gender and ...",NaN,Edinburgh University Press,Byron and the Margins of Romanticism,NaN,NaN,207,NaN,978 1 4744 3941 1,NaN,NaN,NaN,December,2018,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Out of scope for open access requirements,NaN,NaN,NaN,NaN,NaN,62498e62-48e0-4ccc-8aec-ccbfb72ace06,0,0,0
3,10007788,University of Cambridge,D,33,"Music, Drama, Dance, Performing Arts, Film and...",NaN,NaN,NaN,D,' . . . nur ein Gleichnis': Heinrich Schenker ...,NaN,NaN,Music and Letters,100,2,271,NaN,NaN,1477-4631,10.1093/ml/gcz021,NaN,August,2019,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Compliant,NaN,NaN,NaN,NaN,NaN,ff3324db-912d-4a3b-97fd-547881617986,0,0,0
4,10004775,Norwich University of the Arts,D,32,"Art and Design: History, Practice and Theory",NaN,NaN,NaN,C,âBuilding with Light. Spatial Qualities of t...,NaN,Mendrisio Academy Press / Silvana Editoriale C...,Manipulating light in premodern times / Manipo...,NaN,NaN,NaN,NaN,9788836627219,NaN,NaN,NaN,NaN,2014,https://www.researchgate.net/profile/Vladimir-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A - Created and Contested Territories,Out of scope for open access requirements,NaN,NaN,NaN,NaN,NaN,454d1cdb-06f9-47e8-bf91-cbaf9029ef73,0,1,0


In [ ]:
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

import numpy as np

def calculate_weighted_average(per_of_4_stars, per_of_3_stars, per_of_2_stars, per_of_1_stars, per_of_undefined):
    
    
    star_counts = np.array([per_of_4_stars, per_of_3_stars, per_of_2_stars, per_of_1_stars, per_of_undefined])/ 100
    
    star_values = np.array([4, 3, 2, 1, 0])
    
    return np.dot(star_counts, star_values)  


def calculate_variance(per_of_4_stars, per_of_3_stars, per_of_2_stars, per_of_1_stars, per_of_undefined, mean_grade):
    
    star_counts = np.array([per_of_4_stars, per_of_3_stars, per_of_2_stars, per_of_1_stars, per_of_undefined]) / 100
    star_values = np.array([4, 3, 2, 1, 0])
    variance = np.dot(star_counts, (star_values - mean_grade) ** 2)
    return variance


def get_5_class_labels(mean_grade):
    
    return round(mean_grade)
    

def get_9_class_labels(mean_grade):

    return round(mean_grade * 2)


df['Weighted Avg'] = df.apply(lambda row: calculate_weighted_average(row['4*'], row['3*'], row['2*'], row['1*'], row['Unclassified']), axis=1)

df['Variance'] = df.apply(lambda row: calculate_variance(row['4*'], row['3*'], row['2*'], row['1*'], row['Unclassified'], row['Weighted Avg']), axis=1)


df['5 Class Labels'] =  df.apply(lambda row: get_5_class_labels(row['Weighted Avg']), axis = 1)

df['9 Class Labels'] =  df.apply(lambda row: get_9_class_labels(row['Weighted Avg']), axis = 1)





In [ ]:
selected_columns = ["Institution name", "Unit of assessment name", "Number of doctoral degrees awarded in academic year 2013", "Number of doctoral degrees awarded in academic year 2014", "Number of doctoral degrees awarded in academic year 2015", "Number of doctoral degrees awarded in academic year 2016", "Number of doctoral degrees awarded in academic year 2017", "Number of doctoral degrees awarded in academic year 2018", "Number of doctoral degrees awarded in academic year 2019"]

ecs_selected_columns = ecs[selected_columns]

merged_df = df.merge(ecs_selected_columns, on=["Institution name", "Unit of assessment name"], how="left")

merged_df = merged_df[merged_df.index.isin(df.index)]


In [ ]:
from bs4 import BeautifulSoup
import markdown


def remove_before_first_double_newline(text):
    position = text.find('\n\n')
    
    if position != -1:
        return text[position+2:]  # +2 to skip the '\n\n' itself
    else:
        return text
    
def remove_html_tags(text):
    # Parse the HTML content
    soup = BeautifulSoup(text, "html.parser")
    # Get the text without HTML tags
    cleaned_text = soup.get_text()
    return cleaned_text

def remove_markdown(text):
    # Convert Markdown to HTML
    html = markdown.markdown(text)
    # Use BeautifulSoup to strip HTML tags
    cleaned_text = remove_html_tags(html)
    return cleaned_text

def remove_html_and_markdown(text):
    # First, remove Markdown formatting
    text_without_markdown = remove_markdown(text)
    # Then, remove any remaining HTML tags
    cleaned_text = remove_html_tags(text_without_markdown)
    return cleaned_text

def remove_citations(text):
    pattern = r'\[.*?\]|\(.*?\)'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text



In [ ]:
def penn_to_wordnet(penn_pos_tag):

    # Function to convert Penn Treebank part-of-speed tags to corresponding WordNet
    
    tag_dictionary = {
        'NN': 'n', 'NNS': 'n', 'NNP': 'n', 'NNPS': 'n',  
        'JJ': 'a', 'JJR': 'a', 'JJS': 'a',               
        'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 
        'RB': 'r', 'RBR': 'r', 'RBS': 'r',              
        'MD': 'v'  }
        
    
    # return the WordNet tag for given Peen Treebank tag, return "n" if none found
    
    try:
        return tag_dictionary[penn_pos_tag[:2]]

    except:
        return "n"
    

stopwords_set = stopwords.words("english")

punctuation_set = set(punctuation)

wnl = WordNetLemmatizer()


def preprocess_text(text):
        
    # account for edge case when text is empty 
        
    if not isinstance(text, str):
        return ""
        
    # Convert all text to lowercase
        
    text = text.lower() 
        
    # Remove characters that are repeated more than twice in a row
        
    text = re.sub(r'(.)\1{2,}', r'\1', text)
        
    # Replace URLs with space 
        
    text = re.sub(r"http\S+"," ", text)
        
    # Remove HTML tags from text
        
    text = re.sub("<.*?>", "", text)
        
    # Ensure that there is space after each period
        
    text = re.sub(r'\.(?![ .])', '. ', text)
        
    # Replace line break tags with space

    text = re.sub(r'<br />', ' ', text)

    # Remove words containing digits
        
    text = re.sub(r'\d+', '', text)
        
    # Remove all numbers
        
    text = re.sub(r'\b(?=.*\d)(?=.*[a-zA-Z]).*?\b', '', text)
    
    # Remove all numbers including floating points
        
    text = re.sub(r'\b\d+(\.\d+)?', '', text)
        
    new_sentences = []
        
    #regexp tokenizer tokenizes sentences 
        
    tokenized_sentences = [regexp_tokenize(sent, pattern = "\s+", gaps = True) for sent in sent_tokenize(text)]

    # Tokenize the cleaned text into sentences, then tokenize each sentence into words while removing spaces
        
    # Find pos tags for each sentence, pos_tag sents is faster than pos_tag 
        
    new_sentences = []
    
    for sentences in pos_tag_sents(tokenized_sentences):
        new_sentence = []
        #iterate through word/tag pair
        for word, tag in sentences:
            # Check if word in punctuation and stopwords list
            if word not in punctuation_set and word not in stopwords_set:
                if word in contractions:
                    # Expand contractions
                    words = contractions[word].split(" ")
                    new_sentence = new_sentence + words
                else:
                    # Get WordNet Pos Tag and lemmatize word
                    wordnet_pos = penn_to_wordnet(tag)
                    lemmatized_word = wnl.lemmatize(word, pos = wordnet_pos)
                    # Remove non-alphanumeric characters
                        
                    lemmatized_word = re.sub(r'[^\w\s]', '', lemmatized_word)
                        
                    #Check if lemmatized word exists and has length greater than 2
                        
                    if lemmatized_word and len(lemmatized_word) > 2:
                        new_sentence.append(lemmatized_word)
            
        if new_sentence:
            new_sentences.append(new_sentence)
        
    return new_sentences





def clean_text(text):
    
    text = remove_before_first_double_newline(text)
    
    text = remove_html_and_markdown(text)
    
    text = remove_citations(text)
    
    text = preprocess_text(text)
    
    return text


from tqdm import tqdm

tqdm.pandas()

#merged_df["Summary of Impact Processed"] = merged_df["1. Summary of the impact"].apply(lambda x: clean_text(x))

merged_df["Details of Impact Processed"] = merged_df["4. Details of the impact"].apply(lambda x: clean_text(x))

merged_df







In [ ]:
def join_sentences(sents_tokenized):

    return ' '.join([' '.join(sentence) for sentence in sents_tokenized])


merged_df["Details of Impact Processed Strings"] = merged_df["Details of Impact Processed"].apply(lambda x: join_sentences(x)) 

merged_df
